In [141]:
import os
import sys

sys.path.append(os.path.abspath(".."))

import pandas as pd
import polars as pl
from pathlib import Path

from linearmodels.panel import PanelOLS
from statsmodels.tsa.stattools import adfuller

OUPUT_PATH = Path("../latex/imgs/res/")
OUPUT_PATH.mkdir(parents=True, exist_ok=True)
OUPUT_TABLES_PATH = Path("../latex/tables/")
OUPUT_TABLES_PATH.mkdir(parents=True, exist_ok=True)
DATA_OUTPUT_PATH = Path("../data/results/")
DATA_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

DUOPOLY_OUPUT_PATH = Path(OUPUT_PATH) / "duopoly"
DUOPOLY_OUPUT_PATH.mkdir(parents=True, exist_ok=True)

In [142]:
df = (
    pl.read_parquet(DATA_OUTPUT_PATH / "all_experiments.parquet")
    .filter((pl.col("num_agents") == 3) & (pl.col("is_symmetric")))
    .with_columns(
        (pl.col("experiment_timestamp").rank("dense")).alias("run_id"),
        (pl.col("agent").rank("dense")).alias("firm_id"),
        pl.col("chosen_price").truediv(pl.col("alpha")).round(4).alias("price"),
    )
    .rename(
        {
            "round": "period",
            "price": "price",
            "agent_prefix_type": "prompt_prefix",
        }
    )
    .with_columns(
        # concat run_id and firm_id to create a unique identifier
        pl.concat_str(["run_id", "firm_id"], separator="_").alias("run_firm_id")
    )
    .select(["run_firm_id", "run_id", "firm_id", "period", "price", "prompt_prefix"])
    .sort(["run_id", "firm_id", "period"])
)
df

run_firm_id,run_id,firm_id,period,price,prompt_prefix
str,u32,u32,i64,f64,str
"""1_1""",1,1,1,2.5,"""P1"""
"""1_1""",1,1,2,1.5,"""P1"""
"""1_1""",1,1,3,2.0,"""P1"""
"""1_1""",1,1,4,1.75,"""P1"""
"""1_1""",1,1,5,1.875,"""P1"""
…,…,…,…,…,…
"""42_3""",42,3,296,1.3375,"""P2"""
"""42_3""",42,3,297,1.3406,"""P2"""
"""42_3""",42,3,298,1.3406,"""P2"""


# Stationarity check
---

- https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html

The null hypothesis of the Augmented Dickey-Fuller is that there is a unit root, with the alternative that there is no unit root. If the pvalue is above a critical size, then we cannot reject that there is a unit root.

The p-values are obtained through regression surface approximation from MacKinnon 1994, but using the updated 2010 tables. If the p-value is close to significant, then the critical values should be used to judge whether to reject the null.

The autolag option and maxlag for it are described in Greene.

In [143]:
# Initialize counters
stationary_count = 0
non_stationary_count = 0

# Initialize a list to collect results
results = []

# Iterate through each firm ID
for firm_id in df["run_firm_id"].unique():
    # Filter series for the current firm_id
    series = (
        df.filter(pl.col("run_firm_id") == firm_id)
        .select("price")
        .to_series()
        .to_list()
    )

    # Run ADF test
    result = adfuller(series, maxlag=22)

    # Interpret the result
    test_statistic = result[0]
    p_value = result[1]

    # Determine stationarity
    if p_value < 0.05:
        stationary = True
        stationary_count += 1
    else:
        stationary = False
        non_stationary_count += 1

    # Append result to list
    results.append(
        {
            "run_firm_id": firm_id,
            "test_statistic": test_statistic,
            "p_value": p_value,
            "stationary": stationary,
        }
    )

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Print summary table
print("Summary of ADF Test Results by run_firm_id:")
print(results_df[["run_firm_id", "stationary"]].groupby("stationary").count())

# Optionally, print additional details
print("\nDetailed Results:")
print(results_df)

# Optionally, you can also print counts
print(f"\nNumber of stationary series: {stationary_count}")
print(f"Number of non-stationary series: {non_stationary_count}")

Summary of ADF Test Results by run_firm_id:
            run_firm_id
stationary             
False                56
True                 70

Detailed Results:
    run_firm_id  test_statistic       p_value  stationary
0           3_1       -5.800484  4.643514e-07        True
1           5_3       -2.643841  8.427043e-02       False
2          22_1      -43.779876  0.000000e+00        True
3          21_3       -2.594750  9.408726e-02       False
4           6_1       -0.489362  8.940757e-01       False
..          ...             ...           ...         ...
121        27_1       -5.381684  3.700517e-06        True
122        29_2      -10.655906  4.535192e-19        True
123        27_3        0.178158  9.710096e-01       False
124        24_2       -5.941627  2.253832e-07        True
125        27_2       -8.143245  1.014808e-12        True

[126 rows x 4 columns]

Number of stationary series: 70
Number of non-stationary series: 56


In [144]:
# Initialize counters
stationary_count = 0
non_stationary_count = 0

# Initialize a list to collect results
results = []

# Iterate through each firm ID
for firm_id in df["run_firm_id"].unique():
    # Filter series for the current firm_id
    series = (
        df.filter(pl.col("run_firm_id") == firm_id)
        .with_columns(
            #   pl.col("price").diff(1).fill_null(0).alias("price_diff")
            (pl.col("price").log())
            .diff(1)
            .fill_null(0)
            .alias("price_diff")  # NOTE! This uses log differences
        )
        .select("price_diff")
        .to_series()
        .to_list()
    )

    # Run ADF test
    result = adfuller(series, maxlag=22)

    # Interpret the result
    test_statistic = result[0]
    p_value = result[1]

    # Determine stationarity
    if p_value < 0.05:
        stationary = True
        stationary_count += 1
    else:
        stationary = False
        non_stationary_count += 1

    # Append result to list
    results.append(
        {
            "run_firm_id": firm_id,
            "test_statistic": test_statistic,
            "p_value": p_value,
            "stationary": stationary,
        }
    )

# Create a DataFrame from results
results_df = pd.DataFrame(results)

# Print summary table
print("Summary of ADF Test Results by run_firm_id:")
print(results_df[["run_firm_id", "stationary"]].groupby("stationary").count())

# Optionally, print additional details
print("\nDetailed Results:")
print(results_df)

# Optionally, you can also print counts
print(f"\nNumber of stationary series: {stationary_count}")
print(f"Number of non-stationary series: {non_stationary_count}")

Summary of ADF Test Results by run_firm_id:
            run_firm_id
stationary             
False                 5
True                121

Detailed Results:
    run_firm_id  test_statistic       p_value  stationary
0           6_2      -24.879145  0.000000e+00        True
1          21_1      -16.082950  5.353922e-29        True
2          18_3      -12.253149  9.439998e-23        True
3           7_3      -11.846522  7.377267e-22        True
4          37_1       -7.316531  1.225149e-10        True
..          ...             ...           ...         ...
121        35_3      -11.851804  7.179251e-22        True
122         9_3       -3.584681  6.060737e-03        True
123        42_2      -85.942115  0.000000e+00        True
124        26_3      -21.846469  0.000000e+00        True
125        14_2      -11.642048  2.135344e-21        True

[126 rows x 4 columns]

Number of stationary series: 121
Number of non-stationary series: 5


We need to work with price differences.

In [145]:
df = df.with_columns(
    (pl.col("price").log())
    .diff(1)
    .over("run_firm_id")
    .fill_null(0)
    .alias("price_log_diff")  # NOTE! This uses log differences
)
df.head()

run_firm_id,run_id,firm_id,period,price,prompt_prefix,price_log_diff
str,u32,u32,i64,f64,str,f64
"""1_1""",1,1,1,2.5,"""P1""",0.0
"""1_1""",1,1,2,1.5,"""P1""",-0.510826
"""1_1""",1,1,3,2.0,"""P1""",0.287682
"""1_1""",1,1,4,1.75,"""P1""",-0.133531
"""1_1""",1,1,5,1.875,"""P1""",0.068993


# Fixed effects regression (trigger strategy)
---

We are interested in the responsiveness of agents to each other since it is a feature of a reward-punishment strategy. We are interested in stickiness since it measures the persistence of such rewards and punishments.

To measure responsiveness and stickiness, we perform a linear regression with the following model:
$$p_{i,r}^t = \alpha_{i,r} + \gamma p_{i,r}^{t-1} + \delta p_{-i,r}^{t-1}+\epsilon_{i,r}^{t}$$

$$\Delta \log(p_{i,r}^t) =   \gamma \Delta \log(p_{i,r}^{t-1}) + \delta \Delta \log(p_{-i,r}^{t-1})+ \Delta \epsilon_{i,r}^{t}$$


where $p_{i,r}^t$ is the price set by the agent $i$ at period $t$ of run $r$ of the experiment, $p_{i,r}^t$ is the price set by competitors at period $t$ of run $r$ and nd $α_{i,r}$ is a firm-run fixed effect.

In [146]:
df

run_firm_id,run_id,firm_id,period,price,prompt_prefix,price_log_diff
str,u32,u32,i64,f64,str,f64
"""1_1""",1,1,1,2.5,"""P1""",0.0
"""1_1""",1,1,2,1.5,"""P1""",-0.510826
"""1_1""",1,1,3,2.0,"""P1""",0.287682
"""1_1""",1,1,4,1.75,"""P1""",-0.133531
"""1_1""",1,1,5,1.875,"""P1""",0.068993
…,…,…,…,…,…,…
"""42_3""",42,3,296,1.3375,"""P2""",-0.002315
"""42_3""",42,3,297,1.3406,"""P2""",0.002315
"""42_3""",42,3,298,1.3406,"""P2""",0.0


In [147]:
df_fe = (
    df.pivot(
        values="price_log_diff",
        index=["run_id", "period", "prompt_prefix"],
        on="firm_id",
    )
    .rename(
        {
            "1": "1_log_diff",
            "2": "2_log_diff",
            "3": "3_log_diff",
        }
    )
    .with_columns(
        [
            pl.col("1_log_diff").shift(1).alias("1_log_diff_lag"),
            pl.col("2_log_diff").shift(1).alias("2_log_diff_lag"),
            pl.col("3_log_diff").shift(1).alias("3_log_diff_lag"),
        ]
    )
    # # Keep only disjoint periods
    .filter(pl.col("period") % 2 == 0)  # Adjust based on how you define disjointness
    # Alternate firms: period % 3 == 1 → firm 1, period % 3 == 2 → firm 2, period % 3 == 0 → firm 3
    .with_columns(
        [
            pl.when(pl.col("period") % 3 == 1)
            .then(pl.col("1_log_diff"))
            .when(pl.col("period") % 3 == 2)
            .then(pl.col("2_log_diff"))
            .otherwise(pl.col("3_log_diff"))
            .alias("price_log_diff"),
            pl.when(pl.col("period") % 3 == 1)
            .then(pl.col("1_log_diff_lag"))
            .when(pl.col("period") % 3 == 2)
            .then(pl.col("2_log_diff_lag"))
            .otherwise(pl.col("3_log_diff_lag"))
            .alias("price_log_diff_lag_own"),
            pl.when(pl.col("period") % 3 == 1)
            .then((pl.col("2_log_diff_lag") + pl.col("3_log_diff_lag")).mean())
            .when(pl.col("period") % 3 == 2)
            .then((pl.col("1_log_diff_lag") + pl.col("3_log_diff_lag")).mean())
            .otherwise((pl.col("1_log_diff_lag") + pl.col("2_log_diff_lag")).mean())
            .alias("price_log_diff_lag_comp_avg"),
            # pl.when(pl.col("period") % 3 == 1)
            # .then(pl.col("2_log_diff_lag"))
            # .when(pl.col("period") % 3 == 2)
            # .then(pl.col("1_log_diff_lag"))
            # .otherwise(pl.col("1_log_diff_lag"))
            # .alias("price_log_diff_lag_comp_1"),
            # pl.when(pl.col("period") % 3 == 1)
            # .then(pl.col("3_log_diff_lag"))
            # .when(pl.col("period") % 3 == 2)
            # .then(pl.col("3_log_diff_lag"))
            # .otherwise(pl.col("2_log_diff_lag"))
            # .alias("price_log_diff_lag_comp_2"),
        ]
    )
    .sort(["run_id", "period", "prompt_prefix"])
)

df_fe

run_id,period,prompt_prefix,1_log_diff,2_log_diff,3_log_diff,1_log_diff_lag,2_log_diff_lag,3_log_diff_lag,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2,"""P1""",-0.510826,-0.133531,0.693147,0.0,0.0,0.0,-0.133531,0.0,-0.003151
1,4,"""P1""",-0.133531,-0.510826,0.287682,0.287682,-0.336472,0.405465,-0.133531,0.287682,-0.002669
1,6,"""P1""",0.013245,0.111226,0.182322,0.068993,0.125163,0.223144,0.182322,0.223144,-0.002267
1,8,"""P1""",-0.013245,0.027399,0.0,0.0,-0.054067,-0.567984,0.027399,-0.054067,-0.003151
1,10,"""P1""",-0.041385,-0.014389,0.0,-0.013423,-0.05557,0.0,-0.041385,-0.013423,-0.002669
…,…,…,…,…,…,…,…,…,…,…,…
42,292,"""P2""",0.0,0.002396,0.0,-0.009153,0.0,0.0,0.0,-0.009153,-0.002669
42,294,"""P2""",0.0,0.0,0.00231,0.001103,-0.002396,-0.00231,0.00231,-0.00231,-0.002267
42,296,"""P2""",0.0,0.0,-0.002315,0.0,0.0,-0.00231,0.0,0.0,-0.003151


In [148]:
df_fe = df_fe.select(
    [
        "run_id",
        "period",
        "prompt_prefix",
        "price_log_diff",
        "price_log_diff_lag_own",
        "price_log_diff_lag_comp_avg",
        # "price_log_diff_lag_comp_1",
        # "price_log_diff_lag_comp_2",
    ]
)
df_fe

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
1,2,"""P1""",-0.133531,0.0,-0.003151
1,4,"""P1""",-0.133531,0.287682,-0.002669
1,6,"""P1""",0.182322,0.223144,-0.002267
1,8,"""P1""",0.027399,-0.054067,-0.003151
1,10,"""P1""",-0.041385,-0.013423,-0.002669
…,…,…,…,…,…
42,292,"""P2""",0.0,-0.009153,-0.002669
42,294,"""P2""",0.00231,-0.00231,-0.002267
42,296,"""P2""",0.0,0.0,-0.003151


In [149]:
df_fe["prompt_prefix"].value_counts()

prompt_prefix,count
str,u32
"""P2""",3150
"""P1""",3150


## P1vsP1

In [150]:
df_fe_p1 = df_fe.filter(pl.col("prompt_prefix") == "P1").sort(["run_id", "period"])
df_fe_p1

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
1,2,"""P1""",-0.133531,0.0,-0.003151
1,4,"""P1""",-0.133531,0.287682,-0.002669
1,6,"""P1""",0.182322,0.223144,-0.002267
1,8,"""P1""",0.027399,-0.054067,-0.003151
1,10,"""P1""",-0.041385,-0.013423,-0.002669
…,…,…,…,…,…
39,292,"""P1""",0.0,0.0,-0.002669
39,294,"""P1""",0.009803,-0.009803,-0.002267
39,296,"""P1""",0.0,-0.000852,-0.003151


In [151]:
df_fe_p1 = df_fe_p1.to_pandas()
df_fe_p1.set_index(["run_id", "period"], inplace=True)
df_fe_p1.head()

prompt_prefix  price_log_diff  price_log_diff_lag_own  \
run_id period                                                         
1      2                 P1       -0.133531                0.000000   
       4                 P1       -0.133531                0.287682   
       6                 P1        0.182322                0.223144   
       8                 P1        0.027399               -0.054067   
       10                P1       -0.041385               -0.013423   

               price_log_diff_lag_comp_avg  
run_id period                               
1      2                         -0.003151  
       4                         -0.002669  
       6                         -0.002267  
       8                         -0.003151  
       10                        -0.002669

In [152]:
# Run PanelOLS with entity effects (fixed effects)
model = PanelOLS.from_formula(
    "price_log_diff ~ price_log_diff_lag_own + price_log_diff_lag_comp_avg + EntityEffects",
    data=df_fe_p1,
).fit(cov_type="robust")
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:         price_log_diff   R-squared:                        0.0169
Estimator:                   PanelOLS   R-squared (Between):              0.4244
No. Observations:                3150   R-squared (Within):               0.0169
Date:                Wed, Jul 02 2025   R-squared (Overall):              0.0205
Time:                        17:28:06   Log-likelihood                    5808.3
Cov. Estimator:                Robust                                           
                                        F-statistic:                      26.829
Entities:                          21   P-value                           0.0000
Avg Obs:                       150.00   Distribution:                  F(2,3127)
Min Obs:                       150.00                                           
Max Obs:                       150.00   F-statistic (robust):             2.9612
                            

## P2vsP2

In [153]:
df_fe_p2 = df_fe.filter(pl.col("prompt_prefix") == "P2").sort(["run_id", "period"])
df_fe_p2

run_id,period,prompt_prefix,price_log_diff,price_log_diff_lag_own,price_log_diff_lag_comp_avg
u32,i64,str,f64,f64,f64
2,2,"""P2""",-0.223144,0.0,-0.003151
2,4,"""P2""",-0.257829,0.09531,-0.002669
2,6,"""P2""",-0.029853,0.04512,-0.002267
2,8,"""P2""",-0.033902,-0.03279,-0.003151
2,10,"""P2""",-0.015748,0.015748,-0.002669
…,…,…,…,…,…
42,292,"""P2""",0.0,-0.009153,-0.002669
42,294,"""P2""",0.00231,-0.00231,-0.002267
42,296,"""P2""",0.0,0.0,-0.003151


In [154]:
df_fe_p2 = df_fe_p2.to_pandas()
df_fe_p2.set_index(["run_id", "period"], inplace=True)
df_fe_p2.head()

prompt_prefix  price_log_diff  price_log_diff_lag_own  \
run_id period                                                         
2      2                 P2       -0.223144                0.000000   
       4                 P2       -0.257829                0.095310   
       6                 P2       -0.029853                0.045120   
       8                 P2       -0.033902               -0.032790   
       10                P2       -0.015748                0.015748   

               price_log_diff_lag_comp_avg  
run_id period                               
2      2                         -0.003151  
       4                         -0.002669  
       6                         -0.002267  
       8                         -0.003151  
       10                        -0.002669

In [155]:
# Run PanelOLS with entity effects (fixed effects)
model = PanelOLS.from_formula(
    "price_log_diff ~ price_log_diff_lag_own + price_log_diff_lag_comp_avg + EntityEffects",
    data=df_fe_p2,
).fit(cov_type="robust")
print(model.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:         price_log_diff   R-squared:                        0.0043
Estimator:                   PanelOLS   R-squared (Between):             -4.9300
No. Observations:                3150   R-squared (Within):               0.0043
Date:                Wed, Jul 02 2025   R-squared (Overall):             -0.0251
Time:                        17:28:06   Log-likelihood                    5051.7
Cov. Estimator:                Robust                                           
                                        F-statistic:                      6.7595
Entities:                          21   P-value                           0.0012
Avg Obs:                       150.00   Distribution:                  F(2,3127)
Min Obs:                       150.00                                           
Max Obs:                       150.00   F-statistic (robust):             1.2587
                            